In [36]:
# Dependencies
import pandas as pd
import json

In [37]:
#Name of json file
json_file = "purchase_data.json"

In [38]:
#Read json file
with open(json_file, 'r') as datafile:
    datastore = json.load(datafile)


In [39]:
#Convert list into DataFrames
df_gamers = pd.DataFrame(datastore)
df_gamers.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [40]:
#Print List of columns in DataFrame
df_gamers.columns

Index(['Age', 'Gender', 'Item ID', 'Item Name', 'Price', 'SN'], dtype='object')

In [41]:
###PLayer Count
#total number of Players
total_num_players = df_gamers["SN"].count()
total_num_players

total_df = pd.DataFrame({
    "Total Number of Players": [total_num_players]
})
total_df

,Total Number of Players
0,780


In [42]:
#Selecting specific Column information for review and calculations.
reduced_column_A = df_gamers.loc[:, ["Item Name","Price"]]
reduced_column_A.head()

,Item Name,Price
0,Bone Crushing Silver Skewer,3.37
1,"Stormbringer, Dark Blade of Ending Misery",2.32
2,Primitive Blade,2.46
3,Final Critic,1.36
4,Stormfury Mace,1.27


In [43]:
###Purchasing Analysis (Total)

#Calculations of Unique Items, No. Purchases, Avg of Purchases and Total Revenue
#into new DataFrame with column formatting
unique_items = len(reduced_column_A["Item Name"].unique())
unique_items

#caluculation og Average Prices
avg_price = reduced_column_A["Price"].mean()
avg_price

#calculation of total number of Purchases
num_purchases_df = reduced_column_A["Item Name"].count()
num_purchases_df

#calculations of total Price
total_price = reduced_column_A["Price"].sum()
total_price

#create a new DataFrame to hold all calculations
purchasing_df = pd.DataFrame({
    "Number of Unique Items": [unique_items],
    "Average Purchase Price": [avg_price],
    "Total Number of Purchases": [num_purchases_df],
    "Total Revenue": [total_price]
})  

#format columns
purchasing_df["Average Purchase Price"] = purchasing_df["Average Purchase Price"].map("${:.2f}".format)
purchasing_df["Total Revenue"] = purchasing_df["Total Revenue"].map("${:.2f}".format)
purchasing_df

purchasing_df[["Number of Unique Items","Average Purchase Price","Total Number of Purchases","Total Revenue"]].head()

,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,179,$2.93,780,$2286.33


In [44]:
###Gender Demographics

#Grouping the DataFrame by "Gender"
gender_group = df_gamers.groupby("Gender")
gender_group["Gender"].count()

#create new DataFrame for Item Name counts
gender_overall = pd.DataFrame(gender_group["Item Name"].count())
gender_overall.head()

#rename column
gender_overall = gender_overall.rename(
    columns={"Item Name": "Total Count"})
gender_overall.head()

#reset Index
gender_overall = gender_overall.reset_index()
gender_overall.head()

#calculations for Gender Percentage
total_gen_players = df_gamers["Gender"].count()
total_gen_players

count_per_gender = gender_group["Gender"].count()
count_per_gender

#create new DataFrame of Gender Percentage
gender_percent = pd.DataFrame((count_per_gender/total_gen_players)*100)
gender_percent.head()

gender_percent = gender_percent.rename(
    columns={"Gender": "Percentage of Players"})
gender_percent.head()

gender_percent = gender_percent.reset_index()
gender_percent.head()

#merge DataFrames into one by Gender
gender_overall = gender_overall.merge(gender_percent, on="Gender")

#format columns
gender_overall = gender_overall[["Gender","Percentage of Players","Total Count"]]
gender_overall.head()

gender_overall["Percentage of Players"] = gender_overall["Percentage of Players"].map("{:.2f}".format)
gender_overall.head()

,Gender,Percentage of Players,Total Count
0,Female,17.44,136
1,Male,81.15,633
2,Other / Non-Disclosed,1.41,11


In [45]:
###Purchasing Analysis (Gender)

#create a new Data Frame for calculations
purch_group = pd.DataFrame(df_gamers)
purch_group.head()

#Calculations for Total Price Purchase
gen_purch_total = purch_group.groupby(["Gender"]).sum()["Price"]


#Calculations for Average Count
gen_avg_count = purch_group.groupby(["Gender"]).mean()["Price"]


#Calculations for Total Gender count
gen_counts = purch_group.groupby(["Gender"]).count()["Price"]

# Calculate Normalized Purchasing

normalized_total = gen_purch_total / gen_counts
normalized_total

# Convert to DataFrame

gender_summary = pd.DataFrame({"Purchase Count": gen_counts,
                                 "Average Purchase Price": gen_avg_count,
                                 "Total Purchase Value": gen_purch_total,
                              "Normalized Totals": normalized_total
                              })

#Format columns
gender_summary["Average Purchase Price"] = gender_summary["Average Purchase Price"].map("${:,.2f}".format)
gender_summary["Total Purchase Value"] = gender_summary["Total Purchase Value"].map("${:,.2f}".format)
gender_summary["Normalized Totals"] = gender_summary["Normalized Totals"].map("${:,.2f}".format)
gender_summary = gender_summary.loc[:, ["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Normalized Totals"]]
gender_summary

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Female,136,$2.82,$382.91,$2.82
Male,633,$2.95,"$1,867.68",$2.95
Other / Non-Disclosed,11,$3.25,$35.74,$3.25


In [46]:
###Age Demographics


#create new DataFrame to support Bins and groups
BIN_df = pd.DataFrame(df_gamers)
BIN_df

#create table grouped by bins
# Create the bins in which Data will be held over 4 years
bins = [0, 10, 15, 20, 25, 30, 35, 40, 100]
age_ranges = ['<10', '11-15', '16-20', '21-25', '26-30', '31-35', '36-40', '40+']

#create groups to have placed into age_range groups by Age
BIN_df['Age Range'] = pd.cut(BIN_df["Age"], bins, labels=age_ranges)
BIN_df

#Group by Age Range
age_group = BIN_df.groupby("Age Range").count()
age_group["Age"].count()

#create a new DataFrame to store groups
age_overall = pd.DataFrame(age_group["Age"])
age_overall.head()

#rename column as Total Count
age_overall = age_overall.rename(
    columns={"Age": "Total Count"})
age_overall

#reset the Index
age_overall = age_overall.reset_index()
age_overall

#Calculations of Group Precent by Age
total_age_players = BIN_df["Age"].count()
total_age_players

count_per_age = age_group["Age"].count()
count_per_age

age_group_percent = pd.DataFrame((age_group["Age"]/total_age_players)*100)
age_group_percent.head()

#rename column within DataFrame
age_group_percent = age_group_percent.rename(
    columns={"Age": "Percentage of Players"})
age_group_percent

age_group_percent = age_group_percent.reset_index()
age_group_percent

#Merge the Datasets into one by Age Range
age_overall = age_overall.merge(age_group_percent, on="Age Range")

#rearrange columns
age_overall = age_overall[["Age Range","Percentage of Players","Total Count"]]
age_overall.head()

#format column
age_overall["Percentage of Players"] = age_overall["Percentage of Players"].map("{:.2f}".format)
age_overall





,Age Range,Percentage of Players,Total Count
0,<10,4.10,32
1,11-15,10.00,78
2,16-20,23.59,184
3,21-25,39.10,305
4,26-30,9.74,76
5,31-35,7.44,58
6,36-40,5.64,44
7,40+,0.38,3


In [47]:
###Purchasing Analysis by Age

#Calculations for Total Price Purchase
age_total_purchase = BIN_df.groupby(["Age Range"]).sum()["Price"]


#Calculations for Average Count
age_avg_overall = BIN_df.groupby(["Age Range"]).mean()["Price"]


#Calculations for Total Purchase count
age_purchase_count = BIN_df.groupby(["Age Range"]).count()["Price"]



# Calculate Normalized Purchasing
age_normalized_total = age_total_purchase / age_purchase_count
age_normalized_total

# Convert to DataFrame

age_summary = pd.DataFrame({"Purchase Count": age_purchase_count,
                                 "Average Purchase Price": age_avg_overall,
                                 "Total Purchase Value": age_total_purchase,
                              "Normalized Totals": age_normalized_total
                              })

#Format columns
age_summary["Average Purchase Price"] = age_summary["Average Purchase Price"].map("${:,.2f}".format)
age_summary["Total Purchase Value"] = age_summary["Total Purchase Value"].map("${:,.2f}".format)
age_summary["Normalized Totals"] = age_summary["Normalized Totals"].map("${:,.2f}".format)
age_summary = age_summary.loc[:, ["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Normalized Totals"]]
age_summary



,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Age Range,,,,
<10,32,$3.02,$96.62,$3.02
11-15,78,$2.87,$224.15,$2.87
16-20,184,$2.87,$528.74,$2.87
21-25,305,$2.96,$902.61,$2.96
26-30,76,$2.89,$219.82,$2.89
31-35,58,$3.07,$178.26,$3.07
36-40,44,$2.90,$127.49,$2.90
40+,3,$2.88,$8.64,$2.88


In [48]:
###Top Spenders

#group by SN
SN_group = df_gamers.groupby("SN")
SN_group

#Total Price Purchase by SN
top_spend_total_purch = SN_group["Price"].sum()
top_spend_total_purch.head()

#Average Purchase by SN
top_spend_avg_purch = SN_group["Price"].mean()
top_spend_avg_purch.head()

#Total Purchase Count by SN
top_spend_purch_count = SN_group["Price"].count()
top_spend_purch_count.head()

#consolidate all tables
top_spend_summary = pd.DataFrame({"Purchase Count": top_spend_purch_count,
                                 "Average Purchase Price": top_spend_avg_purch,
                                 "Total Purchase Value": top_spend_total_purch})

#format columns and sort them by Total Purchase Value
top_spend_summary["Average Purchase Price"] = top_spend_summary["Average Purchase Price"].map("${:.2f}".format)
top_spend_summary
top_spend_summary["Total Purchase Value"] = top_spend_summary["Total Purchase Value"].map("${:.2f}".format)
top_spend_summary
top_spend_summary = top_spend_summary[["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]
top_spend_summary.sort_values(by=["Total Purchase Value"], ascending=False).head()



,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Qarwen67,4,$2.49,$9.97
Sondim43,3,$3.13,$9.38
Tillyrin30,3,$3.06,$9.19
Lisistaya47,3,$3.06,$9.19
Tyisriphos58,2,$4.59,$9.18


In [49]:
###Popular Items

#Group multiple columns for overall count
grouped_items = df_gamers.groupby(['Item ID', 'Item Name'])
grouped_items.count()

#Total Price Purchase by Item ID and Item Name
item_total_purch = grouped_items["Price"].sum()
item_total_purch.head()

#Average Price Purchase by Item ID and Item Name
item_avg_purch = grouped_items["Price"].mean()
item_avg_purch.head()

#Total Purchase Count by Item ID and Item Name
item_purch_count = grouped_items["Price"].count()
item_purch_count.head()

#create a new DataFrame with all above variables
item_summary = pd.DataFrame({"Purchase Count": item_purch_count,
                                 "Item Price": item_avg_purch,
                                 "Total Purchase Value": item_total_purch})

#format columns and sort by Purchase Count
item_summary["Item Price"] = item_summary["Item Price"].map("${:.2f}".format)
item_summary
item_summary["Total Purchase Value"] = item_summary["Total Purchase Value"].map("${:.2f}".format)
item_summary
item_summary = item_summary[["Purchase Count", "Item Price", "Total Purchase Value"]]
item_summary.sort_values(by=["Purchase Count"], ascending=False).head()



,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
84,Arcane Gem,11,$2.23,$24.53
31,Trickster,9,$2.07,$18.63
175,Woeful Adamantite Claymore,9,$1.24,$11.16
13,Serenity,9,$1.49,$13.41


In [50]:
###Most Profitable Items

#Use same DataFrame from above to create same output of table information
item_summary = pd.DataFrame({"Purchase Count": item_purch_count,
                                 "Item Price": item_avg_purch,
                                 "Total Purchase Value": item_total_purch})

#Format columns and sort by Total Purchase Value
item_summary["Item Price"] = item_summary["Item Price"].map("${:.2f}".format)
item_summary
item_summary["Total Purchase Value"] = item_summary["Total Purchase Value"].map("${:.2f}".format)
item_summary
item_summary = item_summary[["Purchase Count", "Item Price", "Total Purchase Value"]]
item_summary.sort_values(by=["Total Purchase Value"], ascending=False).head()


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
170,Shadowsteel,5,$1.98,$9.90
21,Souleater,3,$3.27,$9.81
37,"Shadow Strike, Glory of Ending Hope",5,$1.93,$9.65
127,"Heartseeker, Reaver of Souls",3,$3.21,$9.63
120,Agatha,5,$1.91,$9.55
